In [1]:
from gridstatus import Ercot
import pandas as pd
ercot = Ercot()

In [2]:
def get_spred_df(year):
    # Fetch data (your code)
    df_rtm = ercot.get_rtm_spp(year=year)  # ~80k rows, 15-min data
    df_dam = ercot.get_dam_spp(year=year)  # ~13k rows, hourly data

    # Step 1: Inspect unique locations (pick one, e.g., 'HB_NORTH' for North Hub)
    print(df_rtm['Location'].unique())  # Outputs array of all settlement points
    location = 'HB_NORTH'  # Or 'LZ_NORTH', 'HB_BUSAVG', etc. – choose based on your strategy

    # Step 2: Filter to selected location
    df_rtm_loc = df_rtm[df_rtm['Location'] == location].copy()
    df_dam_loc = df_dam[df_dam['Location'] == location].copy()

    # Step 3: Parse datetimes (handles -06:00 offset automatically)
    df_rtm_loc['Interval Start'] = pd.to_datetime(df_rtm_loc['Interval Start'])
    df_dam_loc['Interval Start'] = pd.to_datetime(df_dam_loc['Interval Start'])

    # Step 4: Set index to 'Interval Start' for time-series ops
    df_rtm_loc = df_rtm_loc.set_index('Interval Start')
    df_dam_loc = df_dam_loc.set_index('Interval Start')

    # Step 5: Aggregate RTM to hourly (average 4x 15-min prices per hour)
    # Resample to 'H' (hourly), mean of 'Settlement Point Price'
    df_rtm_hourly = df_rtm_loc['SPP'].resample('H').mean().to_frame(name='rt_price')

    # Step 6: Select DA prices (already hourly)
    df_dam_hourly = df_dam_loc['SPP'].to_frame(name='da_price')

    # Step 7: Merge on hourly index (inner join for matching times)
    df_merged = df_dam_hourly.merge(df_rtm_hourly, left_index=True, right_index=True)

    # Step 8: Calculate spread
    df_merged['spread'] = df_merged['rt_price'] - df_merged['da_price']

    # Output: Save or use for model
    df_merged.to_csv(f'ercot_spreads_{year}_{location}.csv')
    print(df_merged.head())
    return df_merged

In [4]:
for year in range(2017,2026):
    get_spred_df(year)

2025-11-09 09:01:45 - INFO - Fetching document https://www.ercot.com/misapp/servlets/IceDocListJsonWS?reportTypeId=13061&_1762646505
2025-11-09 09:01:45 - INFO - Requesting https://www.ercot.com/misapp/servlets/IceDocListJsonWS?reportTypeId=13061&_1762646505 with {}
2025-11-09 09:02:20 - INFO - Fetching document https://www.ercot.com/misapp/servlets/IceDocListJsonWS?reportTypeId=10008&_1762646540
2025-11-09 09:02:20 - INFO - Requesting https://www.ercot.com/misapp/servlets/IceDocListJsonWS?reportTypeId=10008&_1762646540 with {}
2025-11-09 09:02:21 - INFO - Fetching https://www.ercot.com/misdownload/servlets/mirDownload?doclookupId=1157320353
2025-11-09 09:02:22 - INFO - Fetching document https://www.ercot.com/misapp/servlets/IceDocListJsonWS?reportTypeId=13060&_1762646542
2025-11-09 09:02:22 - INFO - Requesting https://www.ercot.com/misapp/servlets/IceDocListJsonWS?reportTypeId=13060&_1762646542 with {}
2025-11-09 09:02:28 - INFO - Fetching document https://www.ercot.com/misapp/servlet

<StringArray>
[ 'HB_BUSAVG',    'LZ_WEST',  'HB_HUBAVG',   'HB_NORTH', 'HB_HOUSTON',
   'HB_SOUTH',    'HB_WEST',   'LZ_SOUTH',     'LZ_AEN',     'LZ_CPS',
   'LZ_RAYBN', 'LZ_HOUSTON',   'LZ_NORTH',    'LZ_LCRA']
Length: 14, dtype: string
                           da_price  rt_price  spread
Interval Start                                       
2017-01-01 00:00:00-06:00     23.23   23.3575  0.1275
2017-01-01 01:00:00-06:00     21.82   21.4650 -0.3550
2017-01-01 02:00:00-06:00     20.16   20.7350  0.5750
2017-01-01 03:00:00-06:00     20.00   20.2700  0.2700
2017-01-01 04:00:00-06:00     20.24   20.1200 -0.1200


2025-11-09 09:03:05 - INFO - Fetching document https://www.ercot.com/misapp/servlets/IceDocListJsonWS?reportTypeId=10008&_1762646585
2025-11-09 09:03:05 - INFO - Requesting https://www.ercot.com/misapp/servlets/IceDocListJsonWS?reportTypeId=10008&_1762646585 with {}
2025-11-09 09:03:05 - INFO - Fetching https://www.ercot.com/misdownload/servlets/mirDownload?doclookupId=1157320353
2025-11-09 09:03:07 - INFO - Fetching document https://www.ercot.com/misapp/servlets/IceDocListJsonWS?reportTypeId=13060&_1762646587
2025-11-09 09:03:07 - INFO - Requesting https://www.ercot.com/misapp/servlets/IceDocListJsonWS?reportTypeId=13060&_1762646587 with {}
2025-11-09 09:03:14 - INFO - Fetching document https://www.ercot.com/misapp/servlets/IceDocListJsonWS?reportTypeId=10008&_1762646594
2025-11-09 09:03:14 - INFO - Requesting https://www.ercot.com/misapp/servlets/IceDocListJsonWS?reportTypeId=10008&_1762646594 with {}
2025-11-09 09:03:14 - INFO - Fetching https://www.ercot.com/misdownload/servlets/mi

<StringArray>
[ 'HB_BUSAVG',    'LZ_WEST',  'HB_HUBAVG',   'HB_NORTH', 'HB_HOUSTON',
   'HB_SOUTH',    'HB_WEST',   'LZ_SOUTH',     'LZ_AEN',     'LZ_CPS',
   'LZ_RAYBN', 'LZ_HOUSTON',   'LZ_NORTH',    'LZ_LCRA']
Length: 14, dtype: string
                           da_price  rt_price  spread
Interval Start                                       
2018-01-01 00:00:00-06:00     27.23   27.5950  0.3650
2018-01-01 01:00:00-06:00     25.55   27.6900  2.1400
2018-01-01 02:00:00-06:00     25.72   27.4825  1.7625
2018-01-01 03:00:00-06:00     26.75   27.9425  1.1925
2018-01-01 04:00:00-06:00     28.11   28.7875  0.6775


2025-11-09 09:03:49 - INFO - Fetching document https://www.ercot.com/misapp/servlets/IceDocListJsonWS?reportTypeId=10008&_1762646629
2025-11-09 09:03:49 - INFO - Requesting https://www.ercot.com/misapp/servlets/IceDocListJsonWS?reportTypeId=10008&_1762646629 with {}
2025-11-09 09:03:50 - INFO - Fetching https://www.ercot.com/misdownload/servlets/mirDownload?doclookupId=1157320353
2025-11-09 09:03:51 - INFO - Fetching document https://www.ercot.com/misapp/servlets/IceDocListJsonWS?reportTypeId=13060&_1762646631
2025-11-09 09:03:51 - INFO - Requesting https://www.ercot.com/misapp/servlets/IceDocListJsonWS?reportTypeId=13060&_1762646631 with {}
2025-11-09 09:03:58 - INFO - Fetching document https://www.ercot.com/misapp/servlets/IceDocListJsonWS?reportTypeId=10008&_1762646638
2025-11-09 09:03:58 - INFO - Requesting https://www.ercot.com/misapp/servlets/IceDocListJsonWS?reportTypeId=10008&_1762646638 with {}
2025-11-09 09:03:58 - INFO - Fetching https://www.ercot.com/misdownload/servlets/mi

...

<StringArray>
[ 'HB_BUSAVG', 'HB_HOUSTON',   'LZ_RAYBN',    'LZ_WEST',  'HB_HUBAVG',
   'LZ_SOUTH',     'LZ_CPS',     'HB_PAN',   'HB_NORTH',   'LZ_NORTH',
 'LZ_HOUSTON',     'LZ_AEN',    'LZ_LCRA',   'HB_SOUTH',    'HB_WEST']
Length: 15, dtype: string
                           da_price  rt_price   spread
Interval Start                                        
2023-01-01 00:00:00-06:00     10.48   -2.1175 -12.5975
2023-01-01 01:00:00-06:00      9.89   -1.1500 -11.0400
2023-01-01 02:00:00-06:00      8.49   -1.1700  -9.6600
2023-01-01 03:00:00-06:00      7.99   -0.0175  -8.0075
2023-01-01 04:00:00-06:00      8.13   -0.8125  -8.9425


2025-11-09 09:08:38 - INFO - Fetching document https://www.ercot.com/misapp/servlets/IceDocListJsonWS?reportTypeId=10008&_1762646918
2025-11-09 09:08:38 - INFO - Requesting https://www.ercot.com/misapp/servlets/IceDocListJsonWS?reportTypeId=10008&_1762646918 with {}
2025-11-09 09:08:39 - INFO - Fetching https://www.ercot.com/misdownload/servlets/mirDownload?doclookupId=1157320353
2025-11-09 09:08:40 - INFO - Fetching document https://www.ercot.com/misapp/servlets/IceDocListJsonWS?reportTypeId=13060&_1762646920
2025-11-09 09:08:40 - INFO - Requesting https://www.ercot.com/misapp/servlets/IceDocListJsonWS?reportTypeId=13060&_1762646920 with {}
2025-11-09 09:08:47 - INFO - Fetching document https://www.ercot.com/misapp/servlets/IceDocListJsonWS?reportTypeId=10008&_1762646927
2025-11-09 09:08:47 - INFO - Requesting https://www.ercot.com/misapp/servlets/IceDocListJsonWS?reportTypeId=10008&_1762646927 with {}
2025-11-09 09:08:47 - INFO - Fetching https://www.ercot.com/misdownload/servlets/mi

<StringArray>
[ 'HB_BUSAVG', 'HB_HOUSTON',   'LZ_RAYBN',    'LZ_WEST',  'HB_HUBAVG',
   'LZ_SOUTH',     'LZ_CPS',     'HB_PAN',   'HB_NORTH',   'LZ_NORTH',
 'LZ_HOUSTON',     'LZ_AEN',    'LZ_LCRA',   'HB_SOUTH',    'HB_WEST']
Length: 15, dtype: string
                           da_price  rt_price  spread
Interval Start                                       
2024-01-01 00:00:00-06:00     16.31   14.8700 -1.4400
2024-01-01 01:00:00-06:00     17.05   16.0900 -0.9600
2024-01-01 02:00:00-06:00     16.79   16.9950  0.2050
2024-01-01 03:00:00-06:00     17.10   17.9525  0.8525
2024-01-01 04:00:00-06:00     18.87   20.6575  1.7875


c:\Users\qwane\miniforge3\envs\power\Lib\site-packages\gridstatus\ercot.py:1250: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat(all_sheets.values())


ValueError: Cannot convert from timedelta64[ns] to timedelta64[h]. Supported resolutions are 's', 'ms', 'us', 'ns'